In [1]:
import pandas as pd
from footix.models.bayesian import Bayesian
import footix.implied_odds as odds
import numpy as np
from footix.data_io.footballdata import ScrapFootballData
from footix.strategy.kelly_strategies import kelly_portfolio_torch, kelly_shrinkage, bayesian_kelly, realKelly
from footix.strategy.portfolio_management import optimise_portfolio_torch
from footix.strategy.bets import OddsInput, Bet
from typing import Literal
from footix.strategy.select_bets import simple_select_bets, select_matches_posterior
import json

In [2]:
dataset = ScrapFootballData(competition="FRA Ligue 1", season="2024-2025", path ="./data", force_reload=True).get_fixtures()

In [3]:
dataset.head(-1)

,div,date,time,home_team,away_team,fthg,ftag,ftr,hthg,htag,...,b365_caha,pcahh,pcaha,max_cahh,max_caha,avg_cahh,avg_caha,bfecahh,bfecaha,match_id
0,F1,16/08/2024,19:45,Le Havre,Paris SG,1,4,A,0,1,...,2.00,1.95,1.97,1.95,2.02,1.89,1.95,1.95,2.02,Le Havre - Paris SG - 16/08/2024
1,F1,17/08/2024,16:00,Brest,Marseille,1,5,A,1,3,...,2.04,1.91,2.02,1.91,2.08,1.85,2.01,1.90,2.09,Brest - Marseille - 17/08/2024
2,F1,17/08/2024,18:00,Reims,Lille,0,2,A,0,1,...,1.70,2.14,1.81,2.14,1.85,2.07,1.80,2.13,1.86,Reims - Lille - 17/08/2024
3,F1,17/08/2024,20:00,Monaco,St Etienne,1,0,H,1,0,...,1.93,2.00,1.93,2.00,2.01,1.95,1.91,1.95,2.00,Monaco - St Etienne - 17/08/2024
4,F1,18/08/2024,14:00,Auxerre,Nice,2,1,H,1,1,...,2.11,1.82,2.13,1.83,2.17,1.77,2.10,1.84,2.16,Auxerre - Nice - 18/08/2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,F1,17/05/2025,20:00,Marseille,Rennes,4,2,H,3,1,...,1.93,1.96,1.96,2.02,1.97,1.95,1.89,2.05,1.91,Marseille - Rennes - 17/05/2025
301,F1,17/05/2025,20:00,Nantes,Montpellier,3,0,H,2,0,...,1.95,1.96,1.96,1.96,2.01,1.92,1.93,1.98,1.97,Nantes - Montpellier - 17/05/2025
302,F1,17/05/2025,20:00,Nice,Brest,6,0,H,3,0,...,2.05,1.80,2.11,1.83,2.11,1.80,2.05,1.82,2.18,Nice - Brest - 17/05/2025
303,F1,17/05/2025,20:00,Paris SG,Auxerre,3,1,H,0,1,...,2.03,2.00,1.91,2.00,2.03,1.90,1.94,2.00,1.98,Paris SG - Auxerre - 17/05/2025


In [4]:
dataset["home_team"].unique()

array(['Le Havre', 'Brest', 'Reims', 'Monaco', 'Auxerre', 'Angers',
       'Montpellier', 'Toulouse', 'Rennes', 'Paris SG', 'Lyon', 'Lille',
       'St Etienne', 'Lens', 'Nantes', 'Nice', 'Strasbourg', 'Marseille'],
      dtype=object)

In [5]:
model = Bayesian(n_teams=18, n_goals=20)

In [7]:
dataset = dataset.iloc[:-9]
model.fit(X_train=dataset)

  0%|          | 0/3000 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

In [8]:
mapping_dict = {
    'AC Ajaccio': 'Ajaccio',
    'Amiens': 'Amiens',
    'Clermont': 'Clermont',
    'Dunkerque': 'Dunkerque',
    'Grenoble': 'Grenoble',
    'Guingamp': 'Guingamp',
    'Caen': 'Caen',
    'FC Martigues': 'Martigues',
    'Metz': 'Metz',
    'Annecy FC': 'Annecy',
    'Bastia': 'Bastia',
    'Laval': 'Laval',
    'Paris FC': 'Paris FC',
    'Pau': 'Pau FC',
    'Rodez': 'Rodez',
    'Troyes': 'Troyes',
    'Lorient': 'Lorient',
    'Red Star': 'Red Star'
}


In [9]:
fixture = pd.read_csv("odds/ligue1_15-05-2025.csv")
fixture = fixture.iloc[1:10]
# Assuming fixture and odds are already defined DataFrames
proba_match = []
list_odds = []
list_samples = {}
for idx, match in fixture.iterrows():
    print("#" * 50)
    print(f"{match['home_team']:<30} {match['away_team']:<30}")
    # Display probabilities from Poisson prediction
    home_team = mapping_dict.get(match["home_team"], match["home_team"])
    away_team = mapping_dict.get(match["away_team"], match["away_team"])

    probas = model.predict(home_team=home_team, away_team=away_team).return_probas()
    lambda_h, lambda_a = model.get_samples(home_team=home_team, away_team=away_team)
    home_prob = probas[0]
    draw_prob = probas[1]
    away_prob = probas[2]
    print(f"  Probabilities: Home: {home_prob:.2f}, Draw: {draw_prob:.2f}, Away: {away_prob:.2f}")
    # Display odds from the function
    odds_list = [match["H"], match["D"], match["A"]]
    proba_match.append(probas)
    list_odds.append(OddsInput(home_team=match["home_team"], away_team=match["away_team"], odds=odds_list))
    list_samples[list_odds[-1].match_id] = (lambda_h, lambda_a)

    odds_result = odds.power(odds=odds_list)[0]
    print(f"  Odds: Home: {odds_result[0]:.2f}, Draw: {odds_result[1]:.2f}, Away: {odds_result[2]:.2f}")
    print("#" * 50)
proba_match = np.asarray(proba_match)

##################################################
Nice                           Brest                         
  Probabilities: Home: 0.58, Draw: 0.21, Away: 0.21
  Odds: Home: 0.68, Draw: 0.18, Away: 0.14
##################################################
##################################################
Lyon                           Angers                        
  Probabilities: Home: 0.64, Draw: 0.20, Away: 0.16
  Odds: Home: 0.78, Draw: 0.14, Away: 0.09
##################################################
##################################################
St Etienne                     Toulouse                      
  Probabilities: Home: 0.26, Draw: 0.24, Away: 0.50
  Odds: Home: 0.48, Draw: 0.22, Away: 0.29
##################################################
##################################################
Marseille                      Rennes                        
  Probabilities: Home: 0.56, Draw: 0.20, Away: 0.24
  Odds: Home: 0.52, Draw: 0.23, Away: 0.25
###############

In [10]:
enhanced_list_bet = select_matches_posterior(odds_input=list_odds, lambda_samples = list_samples, edge_floor=0.1, prob_edge_threshold=0.7, single_bet_per_game=True)

In [11]:
enhanced_list_bet

[Bet(match_id='Paris SG - Auxerre', market='A', odds=11.5, edge_mean=0.814977288876507, prob_mean=0.157824112076218, edge_std=0.739695875081415, prob_edge_pos=0.8766666666666667, stake=0.0),
 Bet(match_id='St Etienne - Toulouse', market='A', odds=3.15, edge_mean=0.5810182445413864, prob_mean=0.5019105538226624, edge_std=0.33898932140248317, prob_edge_pos=0.9560833333333333, stake=0.0),
 Bet(match_id='Lille - Reims', market='A', odds=7.5, edge_mean=0.5484388781802203, prob_mean=0.20645851709069604, edge_std=0.5230168433060252, prob_edge_pos=0.8575, stake=0.0),
 Bet(match_id='Lyon - Angers', market='A', odds=9.0, edge_mean=0.5375799658187588, prob_mean=0.17084221842430652, edge_std=0.6092803559777633, prob_edge_pos=0.8055, stake=0.0),
 Bet(match_id='Nice - Brest', market='A', odds=6.2, edge_mean=0.3718004240005536, prob_mean=0.22125813290331509, edge_std=0.4889805978840162, prob_edge_pos=0.7658333333333334, stake=0.0),
 Bet(match_id='Nantes - Montpellier', market='A', odds=5.9, edge_mean

In [12]:
selected_bet = kelly_portfolio_torch(list_bets=enhanced_list_bet[:9], lambda_samples=list_samples, bankroll=75, verbose=True, bankroll_cap=0.5, per_bet_cap=0.1)

  0%|          | 0/5000 [00:00<?, ?it/s]

Bankroll used: 38.00 €  •  Possible return: 258.65 €
[Paris SG - Auxerre | A] odds=11.50, edge=0.815, p=0.158, stake=6.00
[St Etienne - Toulouse | A] odds=3.15, edge=0.581, p=0.502, stake=6.00
[Lille - Reims | A] odds=7.50, edge=0.548, p=0.206, stake=6.00
[Lyon - Angers | A] odds=9.00, edge=0.538, p=0.171, stake=5.00
[Nice - Brest | A] odds=6.20, edge=0.372, p=0.221, stake=5.00
[Nantes - Montpellier | A] odds=5.90, edge=0.363, p=0.231, stake=5.00
[Lens - Monaco | D] odds=4.05, edge=0.161, p=0.287, stake=5.00


In [13]:
kelly_shrinkage(list_bets=enhanced_list_bet[:9], lambda_samples=list_samples, bankroll=300, bankroll_cap=0.3, per_bet_cap=0.1) #46.6

[Paris SG - Auxerre | A] odds=11.50, edge=0.815, p=0.158, stake=7.00
[St Etienne - Toulouse | A] odds=3.15, edge=0.581, p=0.502, stake=30.00
[Lille - Reims | A] odds=7.50, edge=0.548, p=0.206, stake=8.00
[Lyon - Angers | A] odds=9.00, edge=0.538, p=0.171, stake=6.00
[Nice - Brest | A] odds=6.20, edge=0.372, p=0.221, stake=8.00
[Nantes - Montpellier | A] odds=5.90, edge=0.363, p=0.231, stake=9.00
[Lens - Monaco | D] odds=4.05, edge=0.161, p=0.287, stake=11.00
Bankroll used: 79.00 €
Possible return: 436.25 €


[Bet(match_id='Paris SG - Auxerre', market='A', odds=11.5, edge_mean=0.814977288876507, prob_mean=0.157824112076218, edge_std=0.739695875081415, prob_edge_pos=0.8766666666666667, stake=7),
 Bet(match_id='St Etienne - Toulouse', market='A', odds=3.15, edge_mean=0.5810182445413864, prob_mean=0.5019105538226624, edge_std=0.33898932140248317, prob_edge_pos=0.9560833333333333, stake=30),
 Bet(match_id='Lille - Reims', market='A', odds=7.5, edge_mean=0.5484388781802203, prob_mean=0.20645851709069604, edge_std=0.5230168433060252, prob_edge_pos=0.8575, stake=8),
 Bet(match_id='Lyon - Angers', market='A', odds=9.0, edge_mean=0.5375799658187588, prob_mean=0.17084221842430652, edge_std=0.6092803559777633, prob_edge_pos=0.8055, stake=6),
 Bet(match_id='Nice - Brest', market='A', odds=6.2, edge_mean=0.3718004240005536, prob_mean=0.22125813290331509, edge_std=0.4889805978840162, prob_edge_pos=0.7658333333333334, stake=8),
 Bet(match_id='Nantes - Montpellier', market='A', odds=5.9, edge_mean=0.363341

In [15]:
optimise_portfolio_torch(list_bets=enhanced_list_bet[:9], bankroll=300, max_fraction=0.3, iters=20_000, lr=0.002, verbose=True)

  0%|          | 0/20000 [00:00<?, ?it/s]


Total stake used: 90.00 (30.0 % of bankroll)

 Possible return 642.6
P(portfolio loss)≈ 0.780%
[Paris SG - Auxerre | A] odds=11.50, edge=0.815, p=0.158, stake=18.00
[St Etienne - Toulouse | A] odds=3.15, edge=0.581, p=0.502, stake=14.00
[Lille - Reims | A] odds=7.50, edge=0.548, p=0.206, stake=14.00
[Lyon - Angers | A] odds=9.00, edge=0.538, p=0.171, stake=13.00
[Nice - Brest | A] odds=6.20, edge=0.372, p=0.221, stake=11.00
[Nantes - Montpellier | A] odds=5.90, edge=0.363, p=0.231, stake=11.00
[Lens - Monaco | D] odds=4.05, edge=0.161, p=0.287, stake=9.00


[Bet(match_id='Paris SG - Auxerre', market='A', odds=11.5, edge_mean=0.814977288876507, prob_mean=0.157824112076218, edge_std=0.739695875081415, prob_edge_pos=0.8766666666666667, stake=18.0),
 Bet(match_id='St Etienne - Toulouse', market='A', odds=3.15, edge_mean=0.5810182445413864, prob_mean=0.5019105538226624, edge_std=0.33898932140248317, prob_edge_pos=0.9560833333333333, stake=14.0),
 Bet(match_id='Lille - Reims', market='A', odds=7.5, edge_mean=0.5484388781802203, prob_mean=0.20645851709069604, edge_std=0.5230168433060252, prob_edge_pos=0.8575, stake=14.0),
 Bet(match_id='Lyon - Angers', market='A', odds=9.0, edge_mean=0.5375799658187588, prob_mean=0.17084221842430652, edge_std=0.6092803559777633, prob_edge_pos=0.8055, stake=13.0),
 Bet(match_id='Nice - Brest', market='A', odds=6.2, edge_mean=0.3718004240005536, prob_mean=0.22125813290331509, edge_std=0.4889805978840162, prob_edge_pos=0.7658333333333334, stake=11.0),
 Bet(match_id='Nantes - Montpellier', market='A', odds=5.9, edge

In [23]:
selected_bet = kelly_portfolio_torch(list_bets=enhanced_list_bet[:9], lambda_samples=list_samples, bankroll=300, verbose=True, bankroll_cap=0.3, per_bet_cap=0.1)

  0%|          | 0/5000 [00:00<?, ?it/s]

Bankroll used: 91.00 €  •  Possible return: 614.90 €
[Paris SG - Auxerre | A] odds=11.50, edge=0.814, p=0.158, stake=13.00
[St Etienne - Toulouse | A] odds=3.15, edge=0.578, p=0.501, stake=13.00
[Lille - Reims | A] odds=7.50, edge=0.551, p=0.207, stake=13.00
[Lyon - Angers | A] odds=9.00, edge=0.536, p=0.171, stake=13.00
[Nice - Brest | A] odds=6.20, edge=0.372, p=0.221, stake=13.00
[Nantes - Montpellier | A] odds=5.90, edge=0.357, p=0.230, stake=13.00
[Lens - Monaco | D] odds=4.05, edge=0.161, p=0.287, stake=13.00


In [25]:
model.predict(home_team="Lens", away_team="Monaco").return_probas()

(0.22669227552077034, 0.2918691766283236, 0.481438547850906)

In [22]:
import numpy as np
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.integer, np.int_)):
            return int(obj)
        elif isinstance(obj, (np.floating, np.float_)):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return super().default(obj)

In [15]:
bets_dict_list = [bet.to_dict() for bet in selected_bet]


with open("bets.json", "w") as f:
    json.dump(bets_dict_list, f, indent=4, cls=NumpyEncoder)

In [16]:
#pm.summary(model.trace)fixture = pd.read_csv("odds/ligue1_01-05-2025.csv")
fixture = fixture.iloc[:9]
# Assuming fixture and odds are already defined DataFrames
proba_match = []
list_odds = []
list_samples = {}
for idx, match in fixture.iterrows():
    print("#" * 50)
    print(f"{match['home_team']:<30} {match['away_team']:<30}")
    # Display probabilities from Poisson prediction
    home_team = mapping_dict.get(match["home_team"], match["home_team"])
    away_team = mapping_dict.get(match["away_team"], match["away_team"])
    goals_matrix = model.predict(home_team=home_team, away_team=away_team)

    probas = model.predict(home_team=home_team, away_team=away_team).return_probas()
    lambda_h, lambda_a = model.get_samples(home_team=home_team, away_team=away_team)
    home_prob = probas[0]
    draw_prob = probas[1]
    away_prob = probas[2]
    print(f"  Probabilities: Home: {home_prob:.2f}, Draw: {draw_prob:.2f}, Away: {away_prob:.2f}")
    # Display odds from the function
    odds_list = [match["H"], match["D"], match["A"]]
    proba_match.append(probas)
    list_odds.append(OddsInput(home_team=match["home_team"], away_team=match["away_team"], odds=odds_list))
    list_samples[list_odds[-1].match_id] = (lambda_h, lambda_a)

    odds_result = odds.shin(odds=odds_list)[0]
    print(f"  Odds: Home: {odds_result[0]:.2f}, Draw: {odds_result[1]:.2f}, Away: {odds_result[2]:.2f}")
    print("#" * 50)
proba_match = np.asarray(proba_match)

##################################################
Nice                           Brest                         
  Probabilities: Home: 0.58, Draw: 0.21, Away: 0.21
  Odds: Home: 0.67, Draw: 0.18, Away: 0.14
##################################################
##################################################
Lyon                           Angers                        
  Probabilities: Home: 0.64, Draw: 0.20, Away: 0.16
  Odds: Home: 0.76, Draw: 0.14, Away: 0.09
##################################################
##################################################
St Etienne                     Toulouse                      
  Probabilities: Home: 0.26, Draw: 0.24, Away: 0.50
  Odds: Home: 0.48, Draw: 0.23, Away: 0.29
##################################################
##################################################
Marseille                      Rennes                        
  Probabilities: Home: 0.56, Draw: 0.20, Away: 0.24
  Odds: Home: 0.52, Draw: 0.23, Away: 0.25
###############

In [17]:
model.predict(home_team="Strasbourg", away_team="Paris SG").return_probas()

(0.29720325648114376, 0.21194624442558677, 0.4908504990931116)

In [18]:
model.predict(home_team="Strasbourg", away_team="Paris SG").asian_handicap_results(-2)

(0.054144392926241874, 0.08611302024876438, 0.8597425868248362)